In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.200d.txt
/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.50d.txt
/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.100d.txt
/kaggle/input/web3-crypto-dataset/web3_tweets_1210.csv


In [3]:
!pip install ntscraper


In [7]:
import re
import string 
from ntscraper import Nitter
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import emoji
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Dropout, Bidirectional, SpatialDropout1D
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import accuracy_score, classification_report

## Tweet Data Scraped with Nitter

In [6]:
scraper = Nitter()
crypto_tweets = scraper.get_tweets("crypto", mode='hashtag', number =1200)

Testing instances: 100%|██████████| 6/6 [00:04<00:00,  1.46it/s]


In [7]:
print(crypto_tweets.keys())

dict_keys(['tweets', 'threads'])


In [8]:
print(crypto_tweets['tweets'][0])

{'link': 'https://twitter.com/Kev_Capital_TA/status/1903075312470007876#m', 'text': 'Rate cut projections have went down significantly after the FOMC per the Fed WatchTool. Markets reacting appropriately to the uncertainty. Volatile and erratic due to low liquidity and big money staying sidelined for now. #BTC #Crypto #Altcoins', 'user': {'name': 'Kevin', 'username': '@Kev_Capital_TA', 'profile_id': '1898866059072946176', 'avatar': 'https://pbs.twimg.com/profile_images/1898866059072946176/KonQZDYm_bigger.jpg'}, 'date': 'Mar 21, 2025 · 1:24 PM UTC', 'is-retweet': True, 'is-pinned': False, 'external-link': '', 'replying-to': [], 'quoted-post': {}, 'stats': {'comments': 8, 'retweets': 5, 'quotes': 0, 'likes': 130}, 'pictures': [], 'videos': [], 'gifs': []}


In [9]:
data = {
    'link': [],
    'text': [],
    'user': [],
    'likes': [],
    'retweets': [],
    'comments': []

}

for tweet in crypto_tweets['tweets']:
    data['link'].append(tweet['link'])
    data['text'].append(tweet['text'])
    data['user'].append(tweet['user']['name'])
    data['likes'].append(tweet['stats']['likes'])
    data['retweets'].append(tweet['stats']['retweets'])
    data['comments'].append(tweet['stats']['comments'])

In [10]:
df = pd.DataFrame(data)
df.head()

,link,text,user,likes,retweets,comments
0,https://twitter.com/Kev_Capital_TA/status/1903...,Rate cut projections have went down significan...,Kevin,130,5,8
1,https://twitter.com/SuntronMascot/status/19032...,Chill like Suntron and let success roll in lik...,SUNTRON (TRON MASCOT),0,0,0
2,https://twitter.com/professor_223/status/19031...,$SHATS isn't slowing down! #token burns are ro...,$SHATS PROFESSOR💙,18,9,2
3,https://twitter.com/ThinkingCrypto1/status/190...,🚨COINBASE IS ABOUT TO MAKE CRYPTO HISTORY WITH...,Tony Edward (Thinking Crypto Podcast),1,0,0
4,https://twitter.com/Continental3420/status/190...,A Storm Will Break 🌪️ The Earth Will Shake Tho...,Continental,29,10,1


In [11]:
df.tail()

,link,text,user,likes,retweets,comments
406,https://twitter.com/ApeCryptos/status/19032530...,Shill me your easy 10000x #crypto gems,Crypto Monkey 🐒,28,5,35
407,https://twitter.com/MrStealth27/status/1903275...,Excuses don’t pay bills—action does—so what’s ...,Mr Stealth,2,1,0
408,https://twitter.com/TheBattery_Ton/status/1903...,Grind your way up the leaderboard and hold the...,Battery BOT Official,329,236,47
409,https://twitter.com/CryptoRove43107/status/190...,Transacting value without intermediaries is re...,Crypto Rover 🦈 Gem Hunter  🌐,2,1,1
410,https://twitter.com/abovetopsecret/status/1902...,Any chance we'll see some Zenon action on OKX ...,🤫,23,5,1


In [13]:
crypto_df = df

In [15]:
web3_scraper = Nitter()
web3_tweets = web3_scraper.get_tweets("web3", mode='hashtag', number =1200)

Testing instances: 100%|██████████| 6/6 [00:04<00:00,  1.25it/s]


In [17]:
data = {
    'link': [],
    'text': [],
    'user': [],
    'likes': [],
    'retweets': [],
    'comments': []

}

for tweet in web3_tweets['tweets']:
    data['link'].append(tweet['link'])
    data['text'].append(tweet['text'])
    data['user'].append(tweet['user']['name'])
    data['likes'].append(tweet['stats']['likes'])
    data['retweets'].append(tweet['stats']['retweets'])
    data['comments'].append(tweet['stats']['comments'])

In [19]:
web3_df = pd.DataFrame(data)
web3_df.head()

,link,text,user,likes,retweets,comments
0,https://twitter.com/paparazzitoken/status/1902...,📢 Paparazzi Token(Coming Soon) 📝 Blog posts mi...,Paparazzi Token,1631,2666,532
1,https://twitter.com/cryptoscali/status/1903281...,Web3 is built differently. ❌ No bosses. ❌ No g...,OkayBeary.sol,0,0,0
2,https://twitter.com/FlowTokenn/status/19031604...,🐈‍⬛ Welcome to Flow – where finance meets crea...,flow,165,690,2
3,https://twitter.com/TokenToby/status/190229087...,Will they hold steady while the #Trump trade w...,Tobias J. Sperrin,10,7,0
4,https://twitter.com/MavrykDynamics/status/1894...,🔥 Exciting update: Mavryk Dynamics was feature...,Mavryk Dynamics,3288,424,46


In [21]:
web3_df.tail()

,link,text,user,likes,retweets,comments
794,https://twitter.com/Playa2_Crypt/status/190292...,0x1908 Emile Not here to watch. Here to build...,Playa 2,17,10,2
795,https://twitter.com/DivinerProtocol/status/179...,📣 We’re glad to announce partnership with @Ope...,Diviner Protocol | Building Metaverse,18,22,1
796,https://twitter.com/Egmbsc/status/190323273410...,#Web3 #ZKPs #IVN #Whitepaper,不梭不舒服GG,1,0,1
797,https://twitter.com/DollzBroken/status/1903127...,If you all wanna help me a bit running the ser...,Broken Dollz 👑,9,2,2
798,https://twitter.com/Egmbsc/status/190323265970...,#Web3 #ZKPs #IVN #Whitepaper #Collaboration,不梭不舒服GG,1,0,1


In [23]:
df_concat = pd.concat([crypto_df, web3_df], axis=0)
len(df_concat)

1210

In [24]:
df_concat.to_csv('web3_tweets_1210.csv')

## Reading saved data

In [5]:
df_text = pd.read_csv("/kaggle/input/web3-crypto-dataset/web3_tweets_1210.csv")

In [6]:
df_text = df_text[['text']]
df_text.head()

,text
0,Rate cut projections have went down significan...
1,Chill like Suntron and let success roll in lik...
2,$SHATS isn't slowing down! #token burns are ro...
3,🚨COINBASE IS ABOUT TO MAKE CRYPTO HISTORY WITH...
4,A Storm Will Break 🌪️ The Earth Will Shake Tho...


In [8]:
# Cleaning pipeline
df_text.loc[:, 'cleaned_text'] = df_text['text'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+', '', x, flags=re.MULTILINE))
df_text.loc[:, 'cleaned_text'] = df_text['cleaned_text'].apply(lambda x: re.sub(r'@\w+', '', x))
df_text.loc[:, 'cleaned_text'] = df_text['cleaned_text'].apply(lambda x: re.sub(r'#', '', x))
df_text.loc[:, 'cleaned_text'] = df_text['cleaned_text'].str.lower()
df_text.loc[:, 'cleaned_text'] = df_text['cleaned_text'].apply(lambda x: emoji.demojize(x))
df_text.loc[:, 'cleaned_text'] = df_text['cleaned_text'].apply(lambda x: re.sub(r':[a-z_]+:', '', x))
df_text.loc[:, 'cleaned_text'] = df_text['cleaned_text'].apply(lambda x: ' '.join(x.split()))
df_text.loc[:, 'cleaned_text'] = df_text['cleaned_text'].apply(lambda x: re.sub(r'^RT\s+', '', x))
df_text.loc[:, 'cleaned_text'] = df_text['cleaned_text'].apply(lambda x: re.sub(r'(.)\1+', r'\1\1', x))
df_text.loc[:, 'cleaned_text'] = df_text['cleaned_text'].apply(lambda x: re.sub(r'\d+', '', x))
df_text.loc[:, 'cleaned_text'] = df_text['cleaned_text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [12]:
df_text[['cleaned_text']].head()

,cleaned_text
0,rate cut projections have went down significan...
1,chill like suntron and let success roll in lik...
2,shats isnt slowing down token burns are rollin...
3,coinbase is about to make crypto history with ...
4,a storm will break the earth will shake those ...


In [13]:
# Setting stop words
stop_words = set(stopwords.words('english'))
corpus= []
for tweet in tqdm(df_text['cleaned_text']):
    words = [word.lower() for word in word_tokenize(tweet) if ((word.isalpha() == 1) and word not in stop_words)]
    corpus.append(words)

100%|██████████| 1210/1210 [00:00<00:00, 5660.68it/s]


In [15]:
corpus[0]

['rate',
 'cut',
 'projections',
 'went',
 'significantly',
 'fomc',
 'per',
 'fed',
 'watchtool',
 'markets',
 'reacting',
 'appropriately',
 'uncertainty',
 'volatile',
 'erratic',
 'due',
 'low',
 'liquidity',
 'big',
 'money',
 'staying',
 'sidelined',
 'btc',
 'crypto',
 'altcoins']

In [21]:
corpus_strings = [" ".join(tweet[:512]) for tweet in corpus]

In [22]:
corpus_strings[0]

'rate cut projections went significantly fomc per fed watchtool markets reacting appropriately uncertainty volatile erratic due low liquidity big money staying sidelined btc crypto altcoins'

### Using a SOTA Pre-trained Model based on Stanford Sentiment Treebank from HF to assign sentiment labels to train the LSTM model 

In [18]:
from transformers import pipeline
# Load a pre-trained sentiment analysis model from Hugging Face
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")    

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


In [24]:
#Get Sentiment Labels
sentiment_results = sentiment_pipeline(corpus_strings, truncation=True)

In [25]:
# Convert "POSITIVE" -> 1, "NEGATIVE" -> 0
sentiment_labels = np.array([1 if result["label"] == "POSITIVE" else 0 for result in sentiment_results])

In [26]:
print(f"Text: {corpus_strings[0]}")
print(f"Label: {sentiment_results[0]['label']}, Score: {sentiment_results[0]['score']:.4f}")
print(f"Binary Label: {sentiment_labels[0]}")

Text: rate cut projections went significantly fomc per fed watchtool markets reacting appropriately uncertainty volatile erratic due low liquidity big money staying sidelined btc crypto altcoins
Label: NEGATIVE, Score: 0.9978
Binary Label: 0


In [43]:
sentiment_df = pd.DataFrame({
    "text": corpus_strings,
    "sentiment_label": [result["label"] for result in sentiment_results],  # "POSITIVE" or "NEGATIVE"
    "sentiment_score": [result["score"] for result in sentiment_results],  # Confidence score
    "binary_label": sentiment_labels  # 1 (POSITIVE) or 0 (NEGATIVE)
})
sentiment_df.to_csv("sentiment_dataset.csv", index=False)

In [44]:
sentiment_df.head()

,text,sentiment_label,sentiment_score,binary_label
0,rate cut projections went significantly fomc p...,NEGATIVE,0.997799,0
1,chill like suntron let success roll like waves...,POSITIVE,0.869057,1
2,shats isnt slowing token burns rolling meoware...,NEGATIVE,0.973637,0
3,coinbase make crypto history deribit acquisiti...,NEGATIVE,0.996708,0
4,storm break earth shake laugh us today regret ...,POSITIVE,0.672639,1


### Using TensorFlow's TextVectorization layer to tokenize and pad the sequences.

In [28]:
# Define TextVectorization Layer
vocab_size = 10000 
max_length = 50

vectorizer = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=max_length
)
# Adapt vectorizer to tokenized corpus
vectorizer.adapt(corpus_strings)

In [40]:
# Convert corpus to numerical vectors
X = vectorizer(tf.constant(corpus_strings))
y = tf.constant(sentiment_labels, dtype=tf.int32)

In [41]:
#Split percentages
train_split = int(0.70 *  len(corpus_strings))
val_split = int(0.83 * len(corpus_strings)) 


# Split into train, validation, and test sets
X_train, X_val, X_test = X[:train_split], X[train_split:val_split], X[val_split:]
y_train, y_val, y_test = y[:train_split], y[train_split:val_split], y[val_split:]


In [46]:
# Convert to TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(16).prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(16).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(16).prefetch(tf.data.AUTOTUNE)

## Load GloVe Embeddings

In [32]:
glove_path = "/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.100d.txt"
embedding_dim = 100 #Glove 100D

In [33]:
embedding_index = {}
with open(glove_path, "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], dtype="float32") # Embedding Vector
        embedding_index[word] = vectors

In [35]:
# Embedding Matrix (Map vectorized tokens to GloVe)
vocab = vectorizer.get_vocabulary() # Words in the learned vocab
embedding_matrix = np.zeros((vocab_size, embedding_dim)) #Initialize embedding matrix

for i, word in enumerate(vocab):
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector #Assign Glove vector
        
print("Embedding matrix shape:", embedding_matrix.shape)

Embedding matrix shape: (10000, 100)


## Initializing LSTM Model with the GloVe Embeddings

In [36]:
input_layer = Input(shape=(max_length,))

embedding_layer = Embedding(
    input_dim = vocab_size,
    output_dim = embedding_dim,
    embeddings_initializer = tf.keras.initializers.Constant(embedding_matrix),
    trainable=False #
)(input_layer)

lstm_layer = Bidirectional(LSTM(64, return_sequences=False))(embedding_layer)
output_layer = Dense(1, activation="sigmoid")(lstm_layer) # Binary classification

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 50, 100)             │       1,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 128)                 │          84,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,084,609 (4.14 MB)

 Trainable params: 84,609 (330.50 KB)

 Non-trainable params: 1,000,000 (3.81 MB)

## Training Model

In [47]:
# Training the Bi-directional LSTM model

model.fit(train_dataset, validation_data = val_dataset, epochs = 10)

Epoch 1/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7650 - loss: 0.5790 - val_accuracy: 0.7134 - val_loss: 0.4995
Epoch 2/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8450 - loss: 0.3856 - val_accuracy: 0.8089 - val_loss: 0.4218
Epoch 3/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8699 - loss: 0.3302 - val_accuracy: 0.8344 - val_loss: 0.3777
Epoch 4/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9029 - loss: 0.2703 - val_accuracy: 0.8025 - val_loss: 0.4175
Epoch 5/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9198 - loss: 0.2312 - val_accuracy: 0.7962 - val_loss: 0.4819
Epoch 6/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9283 - loss: 0.1993 - val_accuracy: 0.8153 - val_loss: 0.5036
Epoch 7/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9367 - loss: 0.1717 - val_accuracy: 0.7707 - val_loss: 0.6038
Epoch 8/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9312 - loss: 0.1803 - val_accuracy: 0.7134 - val_loss

In [54]:
# Model Evaluation on test dataset
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Length of Test Dataset: {len(X_test)}")

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7600 - loss: 0.7587
Test Accuracy: 0.7718
Length of Test Dataset: 206


## Saving model & vectorizer

In [56]:
model.save("sentiment_lstm_model.h5")

In [57]:
import pickle

#Save vectorizer
with open("text_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)
